In [3]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_community.document_loaders import TextLoader #load the document
from langchain_text_splitters import RecursiveCharacterTextSplitter #for creating chunks from the loaded document
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings


In [4]:
loader = PyPDFLoader("/Users/gabrielaburaglia/Desktop/guide.pdf")
document = loader.load()

Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 87 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)


In [17]:
#Split the document into chunks using text splitters 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(document)

print("Document chunk info:\n")
print(f"Number of document chunks: {len(chunks)}")
print(f"Sample chunk: \n{chunks[12].page_content}\n")

Document chunk info:

Number of document chunks: 21
Sample chunk: 
Secret book safe  You need: Ø A hardcover book Ø Carpet cutter  Time: ca. 30 min, depending on the size of the book Instructions: Fairly easy: Open the book and draw a rectangle on the first page. It should be the size of the secret compartment you want to create (it’s easiest to just cut out all the text body). Cut this rectangle out of all pages but make sure you don’t cut outside the rectangle. In the end, all pages should form a frame for an empty space in the middle (like a very thick empty picture frame). So from outside, it still looks like a book… but who knows what you hide in your bookshelf? You can also cut out multiple smaller compartments and make it a jewelry box. Let the google picture search “secret book box upcycling” inspire you!  
 https://www.ehow.com/way_5697122_easy-crafts-using-household-items.html



In [9]:
#create embeddings using openAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
import os
cwd = os.getcwd()
  
db_dir = os.path.join(cwd,"chroma_db")
print(db_dir)

/Users/gabrielaburaglia/Documents/GitHub/reuse_buddy_M2/chroma_db


In [10]:
#store the embeddings and chunks into Chroma DB
Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_dir)

In [11]:
#setting up the DB for retrieval
embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
vectorDB = Chroma(persist_directory=db_dir,embedding_function=embeddings_used)

In [12]:
#setting up Retriver
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [13]:
def getRetriever(dir):
    """
    dir is the directory of the vector DB
    """
    embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorDB = Chroma(persist_directory=dir,embedding_function=embeddings_used)
    retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    return retriever

In [14]:
def societal_definition(type):
    society_dict = {
        "Growing": "Focused on expansion, development, and continued acceleration.",
        "Collapsing": "Focused on the decline or breakdown of society, leading to a simpler society.",
        "Controlling": "Focused on managing and controlling societal excesses to preserve resources and maintain order, guided by various values and authoritarian control.",
        "Transforming": "Focused on embracing radical changes that reshapes humanity and the environment, leading to a posthuman future."
    }
    return society_dict.get(type,"")

In [15]:
def textGeneration_langChain_RAG(msg,type, retrieverDir):
    """
    msg is the scenario for the story from the pic (hugging face model output);
    type is the sustainability suggestion mode - Growing, Collapsing, Controlling, Transforming
    """
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.2,
        max_tokens=200,
        timeout=None,
        max_retries=2
    )

    defintion = societal_definition(type)

    system_prompt = (
        "You are an advocate of sustainability. With the object given, come up with a way to reuse this object in the future. From the viewpoint of a society that is {suggestion_mode}. A {suggestion_mode} society is defined as one that is: {societal_definition}."
        "Use the following pieces of retrieved context to generate sustainable, upcycled, object reuse ideas. "
        "Keep the idea to less than 200 words."
        "\n\n"
        "{context}"
    )


    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system", 
                system_prompt
            ),
            (
                "human", 
                "{scenario_lang}"
            ),
        ]
    )

    rag_chain = prompt | llm | StrOutputParser()

    retriever = getRetriever(retrieverDir)

    out_message = rag_chain.invoke({
            "context":retriever,
            "suggestion_mode" : type,
            "scenario_lang" : msg,
            "societal_definition": defintion,
        })

    return out_message

In [53]:
scenario = "garden hose" #example output from huggingface model
story = textGeneration_langChain_RAG(scenario,"Collapsing", db_dir)
print(story)

In a collapsing society where resources are scarce and innovation is key to survival, a garden hose can be repurposed in several sustainable ways. First, it can be transformed into a drip irrigation system for small-scale agriculture. By puncturing small holes along the length of the hose and laying it alongside crops, it can efficiently deliver water directly to the roots, conserving water and maximizing crop yield.

Additionally, sections of the hose can be used as durable ties or bindings. They can secure structures, bind bundles of firewood, or even serve as makeshift belts or straps. The flexibility and strength of the hose material make it ideal for these purposes.

Furthermore, garden hoses can be cut into smaller pieces to create protective grips for tools or handles, providing comfort and reducing wear on hands during manual labor. Lastly, they can be used as insulation for pipes or as padding to prevent damage to fragile items during transport.

By creatively reimagining the 